In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Utilities as U, Parsers as P
from mppi.Calculators import Tools
import matplotlib.pyplot as plt
import os
import numpy as np

In [3]:
omp = 1
mpi = 4

# Analysis of the electron-phonon coupling 

We analyze the effects of the electron-phonon coupling in the electronic and optical properties
of bulk Si.

This analysis is base on the web page : https://www.yambo-code.eu/wiki/index.php/Electron_Phonon_Coupling




In [4]:
run_dir = 'Electron_phonon_Si'

## SCF and NSCF calculations

In [5]:
rr = C.RunRules(mpi=mpi,omp_num_threads=omp)
code = C.QeCalculator(rr)
code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler direct


{'scheduler': 'direct',
 'mpi': 4,
 'omp_num_threads': 1,
 'executable': 'pw.x',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

In [6]:
scf_prefix = 'si_scf'
nscf_prefix = 'si_nscf'

First we perform a scf calculation on a 8x8x8 k grid to ensure that the electronic density of the GS is converged.

The convergence threshold is set to 1e-12, this value and k grid are loosend w.r.t. the paramteres used in the original
tutorial to redcue computational time.

Note that, following the Yambo tutorial, the geometry of the lattice is set using ibrav=0 with the explicit values of the cell
parameters. This is done to avoid possible incompatibility between the PW and Yambo conventions for the FCC lattice.

In [7]:
inp= I.PwInput(file='IO_files/Input_for_electron_phonon_Si_tutorial/si.scf.in')
inp.set_prefix(scf_prefix)
inp.set_pseudo_dir('../pseudos')
inp

{'control': {'calculation': "'scf'",
  'verbosity': "'high'",
  'prefix': "'si_scf'",
  'outdir': "'./'",
  'wf_collect': '.true.',
  'pseudo_dir': "'../pseudos'"},
 'system': {'force_symmorphic': '.true.',
  'ibrav': 0,
  'celldm(1)': 5.132,
  'nat': 2,
  'ntyp': 1,
  'ecutwfc': 40.0},
 'electrons': {'diago_full_acc': '.false.', 'conv_thr': '1.0d-15'},
 'ions': {},
 'cell': {},
 'atomic_species': {'Si': [28.085, 'Si_DOJO_PBEsol.upf']},
 'atomic_positions': {'type': 'crystal',
  'values': [['Si', [0.0, 0.0, 0.0]], ['Si', [0.25, 0.25, 0.25]]]},
 'kpoints': {'type': 'automatic',
  'values': ([8.0, 8.0, 8.0], [0.0, 0.0, 0.0])},
 'cell_parameters': {'type': 'alat',
  'values': [[0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0]]},
 'file': 'IO_files/Input_for_electron_phonon_Si_tutorial/si.scf.in'}

In [8]:
results = code.run(input=inp,run_dir=run_dir,name=scf_prefix)
results

create the run_dir folder : 'Electron_phonon_Si'
run command: mpirun -np 4 pw.x -inp si_scf.in > si_scf.log
computation si_scf is running...
computation si_scf ended


Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL


'/home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_scf.save/data-file-schema.xml'

Then we perform a nscf calculation on a smaller 4x4x4 grid with 12 bands.

In [9]:
inp.set_nscf(12)
inp.set_prefix(nscf_prefix)
inp.set_kpoints(points=[4,4,4])
inp

{'control': {'calculation': "'nscf'",
  'verbosity': "'high'",
  'prefix': "'si_nscf'",
  'outdir': "'./'",
  'wf_collect': '.true.',
  'pseudo_dir': "'../pseudos'"},
 'system': {'force_symmorphic': '.false.',
  'ibrav': 0,
  'celldm(1)': 5.132,
  'nat': 2,
  'ntyp': 1,
  'ecutwfc': 40.0,
  'nbnd': 12},
 'electrons': {'diago_full_acc': '.false.', 'conv_thr': 1e-08},
 'ions': {},
 'cell': {},
 'atomic_species': {'Si': [28.085, 'Si_DOJO_PBEsol.upf']},
 'atomic_positions': {'type': 'crystal',
  'values': [['Si', [0.0, 0.0, 0.0]], ['Si', [0.25, 0.25, 0.25]]]},
 'kpoints': {'type': 'automatic', 'values': ([4, 4, 4], [0.0, 0.0, 0.0])},
 'cell_parameters': {'type': 'alat',
  'values': [[0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0]]},
 'file': 'IO_files/Input_for_electron_phonon_Si_tutorial/si.scf.in'}

In [10]:
results = code.run(input=inp,run_dir=run_dir,name=nscf_prefix,source_dir=os.path.join(run_dir,scf_prefix)+'.save')
results

copy source_dir Electron_phonon_Si/si_scf.save in the /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_nscf.save
run command: mpirun -np 4 pw.x -inp si_nscf.in > si_nscf.log
computation si_nscf is running...
computation si_nscf ended


Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL


'/home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_nscf.save/data-file-schema.xml'

In [11]:
data = P.PwParser(results)

Parse file : /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_nscf.save/data-file-schema.xml


In [12]:
kpoints=-data.kpoints
kpoints

array([[-0.   , -0.   , -0.   ],
       [-0.125, -0.125,  0.125],
       [ 0.25 ,  0.25 , -0.25 ],
       [-0.25 , -0.   , -0.   ],
       [ 0.125,  0.375, -0.375],
       [-0.   ,  0.25 , -0.25 ],
       [ 0.5  , -0.   , -0.   ],
       [ 0.5  , -0.25 , -0.   ]])

## Phonon calculations

With this calculation we can compute the dynamical matrix elements and the dvscf files, these quantities are the 
analougous of the GS density for the phonon calculations

In [19]:
rr = C.RunRules(mpi=mpi,omp_num_threads=omp)
code_ph = C.QeCalculator(rr,executable='ph.x')
code_ph.global_options()

Initialize a QuantumESPRESSO calculator with scheduler direct


{'scheduler': 'direct',
 'mpi': 4,
 'omp_num_threads': 1,
 'executable': 'ph.x',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

In [20]:
ph_prefix = 'si_ph' # We can use directly the scf prefix without the source_dir option in the calculator
inp = I.PhInput()
inp.set_prefix(ph_prefix)
inp['inputph']['fildvscf'] = "'si-dvscf'"
inp['inputph']['fildyn'] = "'si.dyn'"
inp['inputph']['electron_phonon'] = "'dvscf'"
inp.set_kpoints(Tools.build_pw_klist(kpoints))
inp

{'inputph': {'verbosity': "'high'",
  'prefix': "'si_ph'",
  'outdir': "'./'",
  'tr2_ph': 1e-12,
  'trans': '.true.',
  'qplot': '.true.',
  'ldisp': '.false.',
  'fildvscf': "'si-dvscf'",
  'fildyn': "'si.dyn'",
  'electron_phonon': "'dvscf'"},
 'kpoints': {'values': [[-0.0, -0.0, -0.0, 1],
   [-0.125, -0.125, 0.125, 1],
   [0.25, 0.25, -0.25, 1],
   [-0.25, -0.0, -0.0, 1],
   [0.125, 0.375, -0.375, 1],
   [-0.0, 0.25, -0.25, 1],
   [0.5, -0.0, -0.0, 1],
   [0.5, -0.25, -0.0, 1]]}}

In [21]:
results = code_ph.run(input=inp,run_dir=run_dir,name=ph_prefix,source_dir=os.path.join(run_dir,scf_prefix)+'.save')

Skip the run of si_ph
The folder /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_ph.save already exists. Source_dir Electron_phonon_Si/si_scf.save not copied


Once that the si.dyn* files and the dvscf files in the _ph0_ have been computed we can calculate the electron-phonon matrix elements. 

In this case we use the nscf computations as starting point for the electronic quantities  

I erase the folder si_ph.save build in the previous calculation and change name to si_ph.log

In [33]:
ph_nscf_prefix = 'si_ph' #
inp = I.PhInput()
inp.set_prefix(ph_nscf_prefix)
inp['inputph']['fildvscf'] = "'si-dvscf'"
inp['inputph']['fildyn'] = "'si.dyn'"
inp['inputph']['electron_phonon'] = "'yambo'"
inp['inputph']['trans'] = "'.false.'"
inp.set_kpoints(Tools.build_pw_klist(kpoints))
inp

{'inputph': {'verbosity': "'high'",
  'prefix': "'si_ph'",
  'outdir': "'./'",
  'tr2_ph': 1e-12,
  'trans': "'.false.'",
  'qplot': '.true.',
  'ldisp': '.false.',
  'fildvscf': "'si-dvscf'",
  'fildyn': "'si.dyn'",
  'electron_phonon': "'yambo'"},
 'kpoints': {'values': [[-0.0, -0.0, -0.0, 1],
   [-0.125, -0.125, 0.125, 1],
   [0.25, 0.25, -0.25, 1],
   [-0.25, -0.0, -0.0, 1],
   [0.125, 0.375, -0.375, 1],
   [-0.0, 0.25, -0.25, 1],
   [0.5, -0.0, -0.0, 1],
   [0.5, -0.25, -0.0, 1]]}}

In [34]:
results = code_ph.run(input=inp,run_dir=run_dir,name=ph_nscf_prefix,source_dir=os.path.join(run_dir,nscf_prefix)+'.save')

delete folder: /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_ph.save
copy source_dir Electron_phonon_Si/si_nscf.save in the /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_ph.save
run command: mpirun -np 4 ph.x -inp si_ph.in > si_ph.log
computation si_ph is running...


--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD
with errorcode 1.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------
[dal-xps:203223] 3 more processes have sent help message help-mpi-api.txt / mpi-abort
[dal-xps:203223] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages


computation si_ph ended


In [ ]:
######################################################################################################

## Computation of the phonon spectrum and of the electron-phonon matrix

This analysis is performed using QuantumESPRESSO.

First we perform a scf computation for the calculation of the electronic density of the GS

In [10]:
scf_prefix = 'si_scf'
nscf_prefix = 'si_nscf'

In [11]:
inp = I.PwInput(file='IO_files/si_scf.in')
inp.set_prefix(scf_prefix)
inp.set_energy_cutoff(60)
inp.set_kpoints(points=[6,6,6])
#inp

In [12]:
rr = C.RunRules(mpi=mpi,omp_num_threads=omp)
code = C.QeCalculator(rr)
code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler direct


{'scheduler': 'direct',
 'mpi': 4,
 'omp_num_threads': 1,
 'executable': 'pw.x',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

In [13]:
results = code.run(input=inp,run_dir=run_dir,name=scf_prefix)

Skip the run of si_scf


In [14]:
data = P.PwParser(results)
kpoints = data.kpoints
kpoints

Parse file : /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_scf.save/data-file-schema.xml


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.66666667e-01,  1.66666667e-01, -1.66666667e-01],
       [-3.33333333e-01,  3.33333333e-01, -3.33333333e-01],
       [ 5.00000000e-01, -5.00000000e-01,  5.00000000e-01],
       [ 0.00000000e+00,  3.33333333e-01,  0.00000000e+00],
       [-1.66666667e-01,  5.00000000e-01, -1.66666667e-01],
       [ 6.66666667e-01, -3.33333333e-01,  6.66666667e-01],
       [ 5.00000000e-01, -1.66666667e-01,  5.00000000e-01],
       [ 3.33333333e-01,  2.77555756e-17,  3.33333333e-01],
       [ 0.00000000e+00,  6.66666667e-01,  0.00000000e+00],
       [ 8.33333333e-01, -1.66666667e-01,  8.33333333e-01],
       [ 6.66666667e-01, -5.55111512e-17,  6.66666667e-01],
       [ 0.00000000e+00, -1.00000000e+00,  0.00000000e+00],
       [ 6.66666667e-01, -3.33333333e-01,  1.00000000e+00],
       [ 5.00000000e-01, -1.66666667e-01,  8.33333333e-01],
       [-3.33333333e-01, -1.00000000e+00,  0.00000000e+00],
       [-1.66666667e-01,  1.66666667e-01

Then we perform a nscf computations that is useful also for the construction of the
screening function.

In [15]:
numbands = 50

inp.set_nscf(numbands,force_symmorphic=True)
inp.set_prefix(nscf_prefix)
inp.set_kpoints(points=[4,4,4])
#inp

In [16]:
results = code.run(input=inp,run_dir=run_dir,name=nscf_prefix,source_dir=os.path.join(run_dir,scf_prefix)+'.save')

Skip the run of si_nscf
The folder /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_nscf.save already exists. Source_dir Electron_phonon_Si/si_scf.save not copied


In [17]:
data = P.PwParser(results)
kpoints = data.kpoints
kpoints

Parse file : /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_nscf.save/data-file-schema.xml


array([[ 0.  ,  0.  ,  0.  ],
       [-0.25,  0.25, -0.25],
       [ 0.5 , -0.5 ,  0.5 ],
       [ 0.  ,  0.5 ,  0.  ],
       [ 0.75, -0.25,  0.75],
       [ 0.5 ,  0.  ,  0.5 ],
       [ 0.  , -1.  ,  0.  ],
       [-0.5 , -1.  ,  0.  ],
       [-0.25,  0.25,  0.25],
       [ 0.5 , -0.5 , -0.5 ],
       [-0.75,  0.75, -0.25],
       [ 0.75, -0.75, -0.25],
       [-0.5 ,  0.5 ,  0.  ]])

### Run of the phonon calculation

In [18]:
rr = C.RunRules(mpi=mpi,omp_num_threads=omp)
code_ph = C.QeCalculator(rr,executable='ph.x')
code_ph.global_options()

Initialize a QuantumESPRESSO calculator with scheduler direct


{'scheduler': 'direct',
 'mpi': 4,
 'omp_num_threads': 1,
 'executable': 'ph.x',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

In [18]:
ph_prefix = 'si_ph' # I can use directly the scf prefix....
inp = I.PhInput()
inp.set_prefix(ph_prefix)
inp['inputph']['fildvscf'] = "'si-dvscf'"
inp['inputph']['fildyn'] = "'si.dyn'"
inp['inputph']['electron_phonon'] = "'dvscf'"
inp.set_kpoints(Tools.build_pw_klist(kpoints))
inp

{'inputph': {'verbosity': "'high'",
  'prefix': "'si_ph'",
  'outdir': "'./'",
  'tr2_ph': 1e-12,
  'trans': '.true.',
  'qplot': '.true.',
  'ldisp': '.false.',
  'fildvscf': "'si-dvscf'",
  'fildyn': "'si.dyn'",
  'electron_phonon': "'dvscf'"},
 'kpoints': {'values': [[-0.0, -0.0, -0.0, 1],
   [-0.125, -0.125, 0.125, 1],
   [0.25, 0.25, -0.25, 1],
   [-0.25, -0.0, -0.0, 1],
   [0.125, 0.375, -0.375, 1],
   [-0.0, 0.25, -0.25, 1],
   [0.5, -0.0, -0.0, 1],
   [0.5, -0.25, -0.0, 1]]}}

In [17]:
results = code_ph.run(input=inp,run_dir=run_dir,name=ph_prefix,source_dir=os.path.join(run_dir,scf_prefix)+'.save')

copy source_dir Electron_phonon_Si/si_scf.save in the /home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/si_ph.save
run command: mpirun -np 4 ph.x -inp si_ph.in > si_ph.log
computation si_ph is running...


Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


computation si_ph ended


In [30]:
results

'/home/marco/Applications/MPPI/sphinx_source/tutorials/Electron_phonon_Si/ph_test.save/data-file-schema.xml'